In [1]:
# Importar librerias necesarias
import os  ### para ver y cambiar directorio de trabajo
import pandas as pd
import numpy as np
import sqlite3 as sql ### para conectarse a BD
from mlxtend.preprocessing import TransactionEncoder
import plotly.graph_objs as go
import plotly.express as px

In [2]:
##### conectarse a BD #######
conn= sql.connect('db_movies')
cur=conn.cursor()

In [3]:
### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',)]

In [4]:
########### traer tabla de BD a python ####
movies= pd.read_sql("""select *  from movies""", conn)
ratings = pd.read_sql('select * from ratings', conn)

In [5]:
#####Exploración inicial #####

### Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados

movies.info()
movies.head()
movies.duplicated().sum()

ratings.info()
ratings.head()
ratings.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


0

In [6]:
##### Descripción base de ratings
###calcular la distribución de calificaciones
cali=pd.read_sql(""" select
                          "rating" as calificación,
                          count(*) as cantidad
                          from ratings
                          group by "rating"
                          order by cantidad desc""", conn)

In [7]:
cali

,calificación,cantidad
0,4.0,26818
1,3.0,20047
2,5.0,13211
3,3.5,13136
4,4.5,8551
5,2.0,7551
6,2.5,5550
7,1.0,2811
8,1.5,1791
9,0.5,1370


In [8]:

data  = go.Bar( x= cali.calificación,y=cali.cantidad, text=cali.cantidad, textposition="outside")
Layout=go.Layout(title="Conteo de calificaciones",xaxis={'title': 'Calificación', 'tickvals': cali.calificación},yaxis={'title':'Conteo'})
go.Figure(data,Layout)

In [9]:
### calcular cada usuario cuantas peliculas calificó
rating_users=pd.read_sql(''' select "userId" as user_id,
                         count(*) as calificación
                         from ratings
                         group by "user_id"
                         order by calificación asc
                         ''',conn )

In [10]:
rating_users

,user_id,calificación
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


In [11]:
fig  = px.histogram(rating_users, x= 'calificación', title= 'Hist frecuencia de número de calificaciones por usario')
fig.show()

In [12]:
rating_users2=pd.read_sql(''' select "userId" as user_id,
                         count(*) as calificación
                         from ratings
                         group by "user_id"
                         having calificación >=20 and calificación <=600
                         order by calificación asc
                         ''',conn )

In [13]:
fig  = px.histogram(rating_users2, x= 'calificación', title= 'Hist frecuencia de número de calificaciones por usario')
fig.show()

In [14]:
rating_movies=pd.read_sql(''' select movieId ,
                         count(*) as calificación
                         from ratings
                         group by "movieId"
                         order by calificación desc limit 10
                         ''',conn )

In [15]:
rating_movies

,movieId,calificación
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
5,260,251
6,480,238
7,110,237
8,589,224
9,527,220


In [16]:
rating_movies = rating_movies.astype({'movieId': 'str'})

In [17]:
data  = go.Bar( x= rating_movies.movieId,y=rating_movies.calificación, text=rating_movies.movieId, textposition="outside")
Layout=go.Layout(title="Conteo de calificaciones",xaxis={'title': 'movieId'},yaxis={'title':'Conteo'})
go.Figure(data,Layout)